# Linear Model [Logistic Regression]

### Import dependencies

In [3]:
import os
import sys
from datetime import datetime as dt

import tensorflow as tf
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

%matplotlib inline

### Load in the data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('../datasets/MNIST/', one_hot=True)

Extracting ../datasets/MNIST/train-images-idx3-ubyte.gz
Extracting ../datasets/MNIST/train-labels-idx1-ubyte.gz
Extracting ../datasets/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../datasets/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
print('Training data   = {:,}'.format(len(data.train.labels)))
print('Testing data    = {:,}'.format(len(data.test.labels)))
print('Validation data =  {:,}'.format(len(data.validation.labels)))

Training data   = 55,000
Testing data    = 10,000
Validation data =  5,000


### Hyperparameters

In [4]:
# Input
image_size = 28
image_shape = (image_size, image_size)
image_size_flat = image_size * image_size
num_classes = 10

# Training
num_iter = 0
val_batch = 50
test_batch = 70
train_batch = 200
learning_rate = 1e-3

## Building the _Computational Graph_

### Placeholder variables

In [5]:
X = tf.placeholder(tf.float32, [None, image_size_flat])
y = tf.placeholder(tf.float32, [None, num_classes])
y_cls = tf.argmax(y, axis=1)

### Trainable variables

In [6]:
W = tf.Variable(tf.truncated_normal(shape=[image_size_flat, num_classes]))
b = tf.Variable(tf.zeros(shape=[num_classes]))

### Forward Propagation

In [7]:
logits = tf.add(tf.matmul(X, W), b)
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)

### Cost function and Gradient Descent

In [8]:
xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
cost = tf.reduce_mean(xentropy)
# Optimizer
global_step = tf.Variable(0, trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cost, global_step=global_step)

### Performance measure

In [9]:
correct = tf.equal(y_pred_cls, y_cls)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Running the Computational Graph

### Define `tf.Session`

In [10]:
sess = tf.Session()

#### _initializing model/global variables_

In [11]:
def init():
    global num_iter
    __init = tf.global_variables_initializer()
    sess.run(__init)
    num_iter = 0
init()  # Initialize to start with

In [ ]:
save_dir = '../saved/logistic-regression'
tensorboard_dir = os.path.join(save_dir, 'tensorboard')
logdir = os.path.join(tensorboard_dir, 'log')

model_dir = os.path.join(save_dir, 'models')
model_path = os.path.join(model_dir, 'model.ckpt')

saver = tf.train.Saver()
writer = tf.summary.FileWriter(logdir=logdir, graph = sess.graph)

In [ ]:
if tf.gfile.Exists(model_dir):
    try:
        print('INFO: Attempting to restore last checkpoint')
        last_ckpt = tf.train.latest_checkpoint(model_dir)
        saver.restore(sess=sess, save_path=last_ckpt)
        print(f'INFO: Successfully restored last checkpoint {last_ckpt}')
    except Exception as e:
        sys.stderr.write(f'ERR: Could not restore checkpoint. {e}')
        sys.stderr.flush()
else:
    tf.gfile.MakeDirs(model_dir)
    print(f'INFO: Created checkpoint dir @ {model_dir}')

### Define Some Helper functions

#### _Train the model_

In [12]:
def train(iterations=1000):
    global num_iter
    for _ in range(iterations):
        X_batch, y_batch = data.train.next_batch(train_batch)
        feed_dict = {X: X_batch, y: y_batch}
        _, i_global = sess.run(train_step, feed_dict=feed_dict)
        sys.stdout.write(f'\r')
        num_iter += 1
    print(80*'=')
    print('\tCompleted {:,} iterations.'.format(num_iter))
    print(80*'=')

#### Run accuracy

In [13]:
def score(test=True, validation=False, use_batch=True):
    print(80*'=')
    print('Accuracy after {:,} iterations'.format(num_iter))
    feed_dict = None
    if test:
        if use_batch:
            X_batch, y_batch = data.test.next_batch(test_batch)
            feed_dict = {X: X_batch, y: y_batch}
        else:
            feed_dict = {X: data.test.images, y: data.test.labels}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        print('Accuracy on test set: {:.02%}'.format(acc))
    if validation:
        if use_batch:
            X_batch, y_batch = data.validation.next_batch(val_batch)
            feed_dict = {X: X_batch, y: y_batch}
        else:
            feed_dict = {X: data.validation.images, y: data.validation.labels}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        print('Accuracy on validation set: {:.02%}'.format(acc))
    print(80*'=')

## Training the model

In [14]:
train(iterations=10)
score(test=True, use_batch=False)

100%|██████████| 10/10 [00:00<00:00, 56.60it/s]


	Completed 10 iterations.
Accuracy after 10 iterations
Accuracy on test set: 10.30%


In [15]:
train(iterations=90)
score(test=True, use_batch=False)

100%|██████████| 90/90 [00:00<00:00, 408.48it/s]

	Completed 100 iterations.
Accuracy after 100 iterations
Accuracy on test set: 23.90%


In [16]:
train(iterations=900)
score(test=True, use_batch=True)

100%|██████████| 900/900 [00:02<00:00, 447.83it/s]

	Completed 1,000 iterations.
Accuracy after 1,000 iterations
Accuracy on test set: 84.29%


In [17]:
train(iterations=9000)
score(test=True, validation=True, use_batch=True)

100%|██████████| 9000/9000 [00:18<00:00, 474.80it/s]

	Completed 10,000 iterations.
Accuracy after 10,000 iterations
Accuracy on test set: 94.29%
Accuracy on validation set: 88.00%


In [18]:
# sess.close()